<a href="https://colab.research.google.com/github/huyminh1115/Trip-Advisor-Hotel-Project/blob/main/Code/FInal_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
    gpu_name = tf.config.list_physical_devices('GPU')[0]
    print("GPU in use:", gpu_name)
else:
    print("GPU is not available.")


GPU is available.
GPU in use: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from wordcloud import WordCloud

# Preprocessing and evaluation
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/processed_data.csv")

In [ ]:
data.head()

,Review,Rating,words,word_count,tokenized,tokenized_words,new_rating,label
0,nice hotel expensive parking got good deal sta...,4,"['nice', 'hotel', 'expensive', 'parking', 'got...",87,nice hotel expensive parking get good deal sta...,"['nice', 'hotel', 'expensive', 'parking', 'get...",average,2
1,ok nothing special charge diamond member hilto...,2,"['ok', 'nothing', 'special', 'charge', 'diamon...",250,ok nothing special charge diamond member hilto...,"['ok', 'nothing', 'special', 'charge', 'diamon...",poor,0
2,nice rooms not 4* experience hotel monaco seat...,3,"['nice', 'rooms', 'not', '4*', 'experience', '...",217,nice room not experience hotel monaco seattle ...,"['nice', 'room', 'not', 'experience', 'hotel',...",average,1
3,"unique, great stay, wonderful time hotel monac...",5,"['unique,', 'great', 'stay,', 'wonderful', 'ti...",89,unique great stay wonderful time hotel monaco ...,"['unique', 'great', 'stay', 'wonderful', 'time...",excellent,2
4,"great stay great stay, went seahawk game aweso...",5,"['great', 'stay', 'great', 'stay,', 'went', 's...",191,great stay great stay go seahawk game awesome ...,"['great', 'stay', 'great', 'stay', 'go', 'seah...",excellent,2


In [6]:
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import numpy as np
import time

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import os
os.environ["WANDB_DISABLED"] = "true"

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize function for text data
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Custom Dataset class for Hugging Face compatibility
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Initialize KFold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Store results for each fold
results = []

# Perform 10-fold cross-validation
for fold_idx, (train_idx, test_idx) in enumerate(kf.split(data['tokenized']), start=1):
    print(f"\nRunning Fold {fold_idx}...")

    # Split data into train and test sets for this fold
    X_train, X_test = data['tokenized'][train_idx], data['tokenized'][test_idx]
    y_train, y_test = data['label'][train_idx], data['label'][test_idx]

    # Tokenize and create datasets
    train_encodings = tokenize_function(X_train.tolist())
    test_encodings = tokenize_function(X_test.tolist())

    train_dataset = CustomDataset(train_encodings, y_train.tolist())
    test_dataset = CustomDataset(test_encodings, y_test.tolist())

    # Initialize BERT model for sequence classification
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,  # Set small value initially with early stopping
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True
    )

    # Initialize Trainer with EarlyStopping
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )

    # Train the model
    start_time = time.time()
    trainer.train()
    elapsed_time = time.time() - start_time

    # Evaluate on train set
    train_predictions = trainer.predict(train_dataset)
    train_preds = np.argmax(train_predictions.predictions, axis=-1)

    train_accuracy = accuracy_score(y_train, train_preds)
    train_f1 = f1_score(y_train, train_preds, average='weighted')
    train_precision = precision_score(y_train, train_preds, average='weighted')
    train_recall = recall_score(y_train, train_preds, average='weighted')

    # Evaluate on test set
    test_predictions = trainer.predict(test_dataset)
    test_preds = np.argmax(test_predictions.predictions, axis=-1)

    test_accuracy = accuracy_score(y_test, test_preds)
    test_f1 = f1_score(y_test, test_preds, average='weighted')
    test_precision = precision_score(y_test, test_preds, average='weighted')
    test_recall = recall_score(y_test, test_preds, average='weighted')

    # Store results for this fold
    results.append({
        "Fold": fold_idx,
        "Train Accuracy": train_accuracy,
        "Train F1": train_f1,
        "Train Precision": train_precision,
        "Train Recall": train_recall,
        "Test Accuracy": test_accuracy,
        "Test F1": test_f1,
        "Test Precision": test_precision,
        "Test Recall": test_recall,
        "Time": elapsed_time
    })

    # Print fold results
    print(f"Fold {fold_idx} Results:")
    print(f"  Train - Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}")
    print(f"  Test  - Accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}")
    print(f"  Time Taken: {elapsed_time:.2f} seconds\n")

# Calculate and display averaged metrics across all folds
avg_metrics = {
    metric: np.mean([result[metric] for result in results])
    for metric in ["Train Accuracy", "Train F1", "Train Precision", "Train Recall",
                   "Test Accuracy", "Test F1", "Test Precision", "Test Recall", "Time"]
}

print("\nAveraged Results Across All Folds:")
print(f"  Train - Accuracy: {avg_metrics['Train Accuracy']:.4f}, F1: {avg_metrics['Train F1']:.4f}, "
      f"Precision: {avg_metrics['Train Precision']:.4f}, Recall: {avg_metrics['Train Recall']:.4f}")
print(f"  Test  - Accuracy: {avg_metrics['Test Accuracy']:.4f}, F1: {avg_metrics['Test F1']:.4f}, "
      f"Precision: {avg_metrics['Test Precision']:.4f}, Recall: {avg_metrics['Test Recall']:.4f}")
print(f"  Average Time Taken per Fold: {avg_metrics['Time']:.2f} seconds")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Epoch,Training Loss,Validation Loss
1,0.379500,0.346030
2,0.349500,0.408181


Fold 9 Results:
  Train - Accuracy: 0.9016, F1: 0.8952, Precision: 0.8922, Recall: 0.9016
  Test  - Accuracy: 0.8770, F1: 0.8706, Precision: 0.8668, Recall: 0.8770
  Time Taken: 982.79 seconds


Running Fold 10...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.428400,0.332327
2,0.469700,0.386425


Fold 10 Results:
  Train - Accuracy: 0.9010, F1: 0.8927, Precision: 0.8902, Recall: 0.9010
  Test  - Accuracy: 0.8770, F1: 0.8709, Precision: 0.8666, Recall: 0.8770
  Time Taken: 968.58 seconds


Averaged Results Across All Folds:
  Train - Accuracy: 0.9141, F1: 0.9069, Precision: 0.9070, Recall: 0.9141
  Test  - Accuracy: 0.8737, F1: 0.8649, Precision: 0.8626, Recall: 0.8737
  Average Time Taken per Fold: 1174.55 seconds
